In [1]:
import Base: iterate, exp, log, sin, cos, tan, +, ^, -, *, /, sqrt, convert, promote_rule, zero,max, isless
using BenchmarkTools

In [2]:
# The elements are `value`, the result of a computation; 
# `dfdy` a storage location where the derivative of some overall 
# computation (`f(value)`) with respect to the output of this element is stored;

# `parent` storing the "parent" locations that have been used to produce the result `value`;
# `backpropagation!` a function that propogates the derivative `dfdy` to the
# parents via `dfdx = dydx * dfdy` where `dydx` is the derivative of this piece
# of the computation, `dfdy` is the derivative of the overall function with
# respect to the output, and `dfdx` is the same for the next level up.
mutable struct Dual{T <: Number} <: Number
    value::T
    dfdy::T
    parent::Union{Dual{T}, Array{Dual{T},1}, Int, Nothing}
    backpropagation!
end
# Dual(n::Integer, d::Float64, parent::Union{Dual{T}, Array{Dual{T},1}, Int, Nothing}, bp!::Any)  where {T <: Number} = Dual(promote(n, d)..., parent, bp!)
# Dual(n::Float64, d::Integer, parent::Union{Dual{T}, Array{Dual{T},1}, Int, Nothing}, bp!::Any)  where {T <: Number} = Dual(promote(n, d)..., parent, bp!)


In [3]:
# x = Dual(2.0, 2, 1, (dfdy, parents) -> nothing)*  Dual(2, 2, 1, (dfdy, parents) -> nothing)
# promote_type(Dual{Real}, Dual{Number})

In [4]:
function convert(::Type{Dual{T}}, x::T) where T <: Number
    Dual(x, zero(T), nothing, (dfdy, parents) -> nothing)
end

# function convert(::Type{Dual{T}}, x::Dual{S}) where {T, S <: Number}
#     Dual(T(x.value), T(x.dfdy), x.parent, x.backpropagation!)
# end


function convert(::Type{Dual{T}}, x::S) where {T, S <: Number}
#     print("2asdsadsda", typeof(x), "\n")
    Dual(T(x), zero(T), nothing, (dfdy, parents) -> nothing)
end

function convert(::Type{Dual{T}}, x::Dual{T}) where T
    x
end


convert (generic function with 190 methods)

In [5]:
function zero(x::Dual{T}) where T
    Dual(zero(T), zero(T), nothing, (dfdy, parents) -> nothing)
end

zero (generic function with 23 methods)

In [6]:
# function promote_rule(::Type{Dual{T}}, ::Type{Dual{S}}) where {T,S}
#     Dual{promote_type(T,S)}
# end
function promote_rule(::Type{Dual{T}}, ::Type{S}) where {T, S <: Number}
    Dual{promote_type(T,S)}
end

function promote_rule(::Type{T}, ::Type{Dual{S}}) where {T <: Number, S}
    Dual{promote_type(T,S)}
end

function promote_rule(::Type{S}, ::Type{Dual{T}}) where {S <: AbstractIrrational, T}
    Dual{promote_type(S, T)}
end


promote_rule (generic function with 127 methods)

In [7]:
function push_parents!(queue::Array{Dual{T}, 1}, ::Nothing) where T
    # Do nothing
end
function push_parents!(queue::Array{Dual{T}, 1}, i::Int) where T
    # Do nothing
end
function push_parents!(queue::Array{Dual{T}, 1}, ls::Array{Dual{T}, 1}) where T
    append!(queue, ls)
end
function push_parents!(queue::Array{Dual{T}, 1}, l::Dual{T}) where T
    push!(queue, l)
end

push_parents! (generic function with 4 methods)

In [8]:
function backprop!(l::Dual{T}) where T
    # Apparently we need this construction because otherwise l gets copied when
    # put into the array.
    backprop!([l])
end
function backprop!(queue::Array{Dual{T},1}) where T
    while length(queue) > 0
        l = popfirst!(queue)
        l.backpropagation!(l.dfdy, l.parent)
        push_parents!(queue, l.parent)
    end
end

backprop! (generic function with 2 methods)

In [9]:
function collect_outputs(l::Dual{T}) where T
    queue = Dual{T}[l]

    outputs = Dual{T}[]

    while length(queue) > 0
        l = popfirst!(queue)
        if typeof(l.parent) <: Int
            push!(outputs, l)
        elseif typeof(l.parent) == Dual{T}
            push!(queue, l.parent)
        elseif typeof(l.parent) == Array{Dual{T}, 1}
            append!(queue, l.parent)
        else # Nothing
            # Do nothing
        end
    end

    outputs
end


collect_outputs (generic function with 1 method)

In [10]:
function derivativeСalculation(f)
    function dfdx(x::T) where T <: Number
        # Pass the function a backward infinitesimal whose backprop function
        # stores the backprop derivative in dfdx_store
#         print("x::T", "\n")

        x = Dual(x, zero(x), 1, (dfdy, parents) -> nothing)

        result = f(x)

        result.dfdy = one(result.value)
        backprop!(result)

        y = collect_outputs(result)[1]

        return y.dfdy
    end

    function dfdx(x::Array{T}, i) where T <: Number
#         print("array", "\n")
        fargs = [Dual(xelt, zero(xelt), i, (dfdy, parents) -> nothing) for (i, xelt) in enumerate(x)]
        result = f(fargs)[i]
#         println(result)
        result.dfdy = one(result.value)
#         println(result)
        backprop!(result)
        y = collect_outputs(result)
        grad = zeros(typeof(result.value), length(x))
        for yelt in y
            grad[yelt.parent] = yelt.dfdy
        end

        return grad
    end

    function dfdx(x...)
        print("x...", "\n")

        fargs = [Dual(xelt, zero(xelt), i, (dfdy, parents) -> nothing) for (i, xelt) in enumerate(x)]
        result = f(fargs...)
        result.dfdy = one(result.value)
        backprop!(result)
        y = collect_outputs(result)

        grad = zeros(typeof(result.value), length(x))
        for yelt in y
            grad[yelt.parent] = yelt.dfdy
        end

        return grad
    end

    return dfdx
end

derivativeСalculation (generic function with 1 method)

In [11]:
function derivativeСalculation(i::Integer, f)
    df = derivativeСalculation(f)
    function df_wrapper(x...)
        g = df(x...)
        return g[i]
    end
    return df_wrapper
end

derivativeСalculation (generic function with 2 methods)

In [12]:
# overloading Plus
function backpropagationForPlus!(dfdy, xy)
    x, y = xy
    x.dfdy += dfdy
    y.dfdy += dfdy
end

function +(x::Dual{T}, y::Dual{T}) where T
#     println("Przecieążenie +")
    if x == y
#         println("x = y +")
        new_y = Dual(y.value, y.dfdy, y.parent, y.backpropagation!)
        return Dual(x.value + y.value, zero(T), [x, new_y], backpropagationForPlus!)
    end
    Dual(x.value + y.value, zero(T), [x, y], backpropagationForPlus!)
end

#--------------- overloading Minus
function backpropagationForMinus!(dfdy, xy) 
    x, y = xy
    x.dfdy += dfdy
    y.dfdy -= dfdy
end

function -(x::Dual{T}, y::Dual{T}) where T
    Dual(x.value - y.value, zero(T), [x, y], backpropagationForMinus!)
end



function backpropagationForMinusOneDual!(dfdy, x)
    x.dfdy -= dfdy
end

function -(x::Dual{T}) where T
    Dual(-x.value, zero(T), x, backpropagationForMinusOneDual!)
end


#------------------- overloading Multiplication

function backpropagationForMultiplication!(dfdy, xy)
    x,y = xy
    
    x.dfdy += dfdy*y.value
    y.dfdy += x.value*dfdy
end


function *(x::Dual{T}, y::Dual{T}) where T
#     println("mnożenie", x == y)
#     println(x)
#     println(y)
    return Dual(x.value*y.value, zero(T), [x,y], backpropagationForMultiplication!)
end


#-------------------- overloading Dividing

function /(x::Dual{T}, y::Dual{T}) where T
    yinv = one(T)/y.value

    function backpropagationForDividing!(dfdy, xy)
        a,b = xy
        a.dfdy += dfdy*yinv
        b.dfdy -= a.value*dfdy*(yinv*yinv)
    end
    if x == y
#         println("x = y /")
        new_y = Dual(y.value, y.dfdy, y.parent, y.backpropagation!)
        return Dual(x.value*yinv, zero(T), [x, new_y], backpropagationForDividing!)
    end
    Dual(x.value*yinv, zero(T), [x,y], backpropagationForDividing!)
end

#-------------------- overloading EXP

function exp(x::Dual{T}) where T
    expValue = exp(x.value)

    function bp!(dfdy, p)
        p.dfdy += dfdy*expValue
    end

    Dual(expValue, zero(expValue), x, bp!)
end

function exp(xs::Array)
    return [exp(x) for x in xs]
end


#-------------------- overloading SIN
function sin(x::Dual{T}) where T
    
    function bp!(dfdy, p)
        p.dfdy += cos(x.value)*dfdy
    end

    sx = sin(x.value)
    Dual(sx, zero(sx), x, bp!)
end

#-------------------- overloading COS
function cos(x::Dual{T}) where T
    
    function bp!(dfdy, p)
        p.dfdy -= sin(x.value)*dfdy
    end

    cx = cos(x.value)
    Dual(cx, zero(cx), x, bp!)
end
#-------------------- overloading TAN
function tan(x::Dual{T}) where T
    c = cos(x.value)
    
    function bp!(dfdy, p)
        p.dfdy += dfdy/(c*c)
    end

    tx = tan(x.value)
    Dual(tx, zero(tx), x, bp!)
end

#---------------------- overloading SQRT
function sqrt(x::Dual{T}) where T
    sqrtValue = sqrt(x.value)

    function bp!(dfdy, p)
        p.dfdy += dfdy/(2*sqrtValue)
    end

    Dual(sqrtValue, zero(sqrtValue), x, bp!)
end

#------------------ overloading ^
function ^(a::Dual{T}, x::Dual{T}) where T
    value = a.value^x.value
#     println("potegowanie daszek")
    function bp!(dfdy, xy)
        a,x = xy
#         print(typeof(dfdy * (x.value) * (a.value)^(x.value - 1)), " nic  ", typeof(dfdy * a.value^x.value*log(a.value)))
        a.dfdy += dfdy * (x.value) * (a.value)^(x.value - 1)
        x.dfdy += dfdy * a.value^x.value*log(a.value)
    end

    Dual(value, zero(value), [a, x], bp!)
end

function ^(x::Dual{T}, n::Integer) where T
    value = x.value^n
#     println("potegowanie daszek22")
    function bp!(dfdy, x)
        x.dfdy += n * dfdy*x.value^(n-1)
    end
    
    Dual(value, zero(value), x, bp!)
end

#------------------ overloading log
function log(x::Dual{T}) where T
    
    function bp!(dfdy, p)
        p.dfdy += dfdy/x.value
    end
    
    Dual(log(x.value), zero(T), x, bp!)
end
#----------------------- overloading max
function max(a, x::Dual{T}) where T
    
    function bp!(dfdy,p)
        p.dfdy += x.value < a ? a : 1 * dfdy
    end
    
    Dual(max(0, x.value), zero(T), x, bp!)
    
end


isless(x::Dual, y::Dual) = x.value < y.value;

In [13]:

# function iterate(iter::Dual, state=1)
# #     print("iterate")
#     if state > length(iter.x)
#         return nothing
#     end
#     return (iter[state],state+1)
# end

function softmax(vector::Array)
    e = exp(vector)
#     e = [MathConstants.e^(x) for x in vector]
#     s = zero(vector[1])
#     z = [Dual(2.0, 1.0, nothing, (dfdy, parents) -> nothing), Dual(21.0, 1.0, nothing, (dfdy, parents) -> nothing)]
#     for x in z
#        s = s + x 
#     end    
#     println(s, sum(e))
    return e / sum(e)
end

softmax (generic function with 1 method)

In [14]:
J = function jacobian(f, number_of_functions, args::Vector{T}) where {T <:Number}
    jacobian_rows = Matrix{T}[]
    d = derivativeСalculation(f)  
    for i=1:number_of_functions
#         x = Dual{T}[]
#         for j=1:length(args)
#             seed = (i == j)
#             push!(x, seed ?
#                 Dual(args[j], one(args[j])) :
#                 Dual(args[j],zero(args[j])) )
#         end
#         temp  = [f(x)..]
        rows = d(args,i)
        push!(jacobian_rows, rows[:,:])
    end
    jacobian_rows
end

jacobian (generic function with 1 method)

In [15]:
function test(a, x)
    return a*x
end

test (generic function with 1 method)

In [16]:
d = derivativeСalculation(test)

(::var"#dfdx#12"{typeof(test)}) (generic function with 3 methods)

In [17]:
d(2, 3)

x...


2-element Vector{Int64}:
 3
 2

In [18]:
# f(x::Vector) = [2x[1]*x[2], 3x[2]*x[3]^2]
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]


f (generic function with 1 method)

In [20]:
# f(x::Vector) = x[1]*x[2]*x[3]
d = derivativeСalculation(f)
d([1.0,2.0,3.0], 5)

3-element Vector{Float64}:
  0.37753914284153406
 -0.0755078285683068
 -0.0755078285683068

In [21]:
J(f,5,[1.,2., 3.]) 


5-element Vector{Matrix{Float64}}:
 [1.0; 0.0; 0.0]
 [0.0; 0.0; -0.5555555555555556]
 [0.0; 16.0; -2.0]
 [1.6209069176044193; 0.0; 0.8414709848078965]
 [0.37753914284153406; -0.0755078285683068; -0.0755078285683068]

In [22]:
J(softmax,2,[1., 2.]) 

2-element Vector{Matrix{Float64}}:
 [0.46555335461147695; -0.19661193324148185]
 [-0.19661193324148188; 0.9276705118714866]

In [23]:
x = [Dual(1.0, 0.0, nothing, nothing), Dual(2.0, 0.0, nothing, nothing), Dual(3.0, 0.0, nothing, nothing)]
y = [Dual(2.0, 0.0, nothing, nothing), Dual(2.0, 0.0, nothing, nothing), Dual(2.0, 0.0, nothing, nothing)]


3-element Vector{Dual{Float64}}:
 Dual{Float64}(2.0, 0.0, nothing, nothing)
 Dual{Float64}(2.0, 0.0, nothing, nothing)
 Dual{Float64}(2.0, 0.0, nothing, nothing)

In [24]:
sum(x)


Dual{Float64}(6.0, 0.0, Dual{Float64}[Dual{Float64}(3.0, 0.0, Dual{Float64}[Dual{Float64}(1.0, 0.0, nothing, nothing), Dual{Float64}(2.0, 0.0, nothing, nothing)], backpropagationForPlus!), Dual{Float64}(3.0, 0.0, nothing, nothing)], backpropagationForPlus!)

In [25]:
sa(x) = sin.(x.*x)


sa (generic function with 1 method)

In [26]:
sa([2, 3])

2-element Vector{Float64}:
 -0.7568024953079282
  0.4121184852417566

In [27]:
#J(s,3, [1., 2., 3.]) 
d = derivativeСalculation(sa)
J(sa,3, [1., 2., 3.]) 

3-element Vector{Matrix{Float64}}:
 [1.0806046117362795; 0.0; 0.0]
 [0.0; -2.6145744834544478; 0.0]
 [0.0; 0.0; -5.466781571308061]

In [28]:
softmax([1,2])


2-element Vector{Float64}:
 0.2689414213699951
 0.7310585786300049

In [28]:
function rosenbrock(x)
    value = zero(x[1])
#     value = (1-x[1])^2 + 100*(x[2] - x[1]^2)^2
    for i=2:length(x)
        value += (1-x[i-1])^2 + 100*(x[i] - x[i-1]^2)^2
    end
    value
end

rosenbrock (generic function with 1 method)

In [29]:
rosenbrocka([1.0,2.0])

LoadError: [91mUndefVarError: rosenbrocka not defined[39m

In [30]:
d = D(rosenbrock)

(::var"#dfdx#12"{typeof(rosenbrock)}) (generic function with 3 methods)

In [37]:
@benchmark J(rosenbrock,1,[rand(1000,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  1.46 MiB
  allocs estimate:  41485
  --------------
  minimum time:     1.957 ms (0.00% GC)
  median time:      2.285 ms (0.00% GC)
  mean time:        2.466 ms (6.31% GC)
  maximum time:     10.538 ms (74.39% GC)
  --------------
  samples:          2022
  evals/sample:     1

In [45]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]
@benchmark J(f,6,[rand(3,1)...]) 

BenchmarkTools.Trial: 
  memory estimate:  18.81 KiB
  allocs estimate:  352
  --------------
  minimum time:     9.757 μs (0.00% GC)
  median time:      10.883 μs (0.00% GC)
  mean time:        14.050 μs (13.66% GC)
  maximum time:     5.139 ms (99.48% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [29]:
f(x::Vector) = [x[1], 5/x[3], 4x[2]^2-2x[3], x[3]*sin(x[1]), exp(x[1])/sum(x), x[1]^2 - x[2]^3 - 88]
J(f,6,[2.9,2.2,1546.23]) 


6-element Vector{Matrix{Float64}}:
 [1.0; 0.0; 0.0]
 [0.0; 0.0; -2.0913263714842668e-6]
 [0.0; 17.6; -2.0]
 [-1501.3246436992515; 0.0; 0.23924932921398243]
 [0.011707650961926682; -7.55171541667044e-6; -7.55171541667044e-6]
 [5.8; -14.520000000000003; 0.0]